In [16]:
import h5py

# Inspect the training dataset
train_dataset = h5py.File('Tr.h5', "r")
print("Training data keys:", list(train_dataset.keys()))

# Inspect the test dataset
test_dataset = h5py.File('Te.h5', "r")
print("Test data keys:", list(test_dataset.keys()))


Training data keys: ['images', 'labels']
Test data keys: ['images', 'labels']


In [17]:
import h5py
import numpy as np
from keras.utils import to_categorical

# Load the training data
train_dataset = h5py.File('Tr.h5', "r")
X_train = np.array(train_dataset['images'][:])  # Use 'images' key for features
Y_train = np.array(train_dataset['labels'][:])  # Use 'labels' key for labels

# Load the test data
test_dataset = h5py.File('Te.h5', "r")
X_test = np.array(test_dataset['images'][:])  # Use 'images' key for features
Y_test = np.array(test_dataset['labels'][:])  # Use 'labels' key for labels

# Flatten the images into vectors
X_train_flatten = X_train.reshape(X_train.shape[0], -1).T
X_test_flatten = X_test.reshape(X_test.shape[0], -1).T

# Standardize the data
X_train = X_train_flatten / 255.
X_test = X_test_flatten / 255.

# One-hot encode the labels
Y_train = to_categorical(Y_train, num_classes=5)
Y_test = to_categorical(Y_test, num_classes=5)

print(f"Number of training examples: {X_train.shape[1]}")
print(f"Number of test examples: {X_test.shape[1]}")
print(f"Each image is of size: {X_train.shape[0]}")


Number of training examples: 250
Number of test examples: 50
Each image is of size: 49152


In [18]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / expZ.sum(axis=0, keepdims=True)


In [19]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)  # Number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters

def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    if activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = softmax(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = np.maximum(0, Z)

    cache = (linear_cache, Z)
    return A, cache

def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2  # Number of layers

    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation="relu")
        caches.append(cache)

    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation="softmax")
    caches.append(cache)
    
    return AL, caches


In [20]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1./m * np.dot(dZ, A_prev.T)
    db = 1./m * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)

    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, Z = cache

    if activation == "relu":
        dZ = np.array(dA, copy=True)
        dZ[Z <= 0] = 0

    elif activation == "softmax":
        dZ = dA  # For softmax and cross-entropy, the gradient is just dA

    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db


In [21]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    print(f"AL shape: {AL.shape}")
    print(f"Y shape: {Y.shape}")
    
    if Y.shape != AL.shape:
        Y = Y.T
    
    cost = -np.sum(Y * np.log(AL + 1e-8)) / m
    return np.squeeze(cost)


In [22]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2  # Number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l+1)]

    return parameters

def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)  # Number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    dAL = AL - Y

    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="softmax")

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation="relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost = False):
    np.random.seed(1)
    costs = []  # keep track of cost
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):

        # Forward propagation
        AL, caches = L_model_forward(X, parameters)
        
        # Compute cost
        cost = compute_cost(AL, Y)
    
        # Backward propagation
        grads = L_model_backward(AL, Y, caches)
 
        # Update parameters
        parameters = update_parameters(parameters, grads, learning_rate)
                
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    return parameters, costs


In [23]:
layers_dims = [X_train.shape[0], 20, 7, 5]  # Example layer dimensions, adjust as needed
parameters, costs = L_layer_model(X_train, Y_train, layers_dims, num_iterations = 2500, print_cost = True)

# Make predictions
def predict(X, parameters):
    AL, _ = L_model_forward(X, parameters)
    predictions = np.argmax(AL, axis=0)
    return predictions

# Predict on training and test sets
train_predictions = predict(X_train, parameters)
test_predictions = predict(X_test, parameters)

# Evaluate performance
train_accuracy = np.mean(train_predictions == np.argmax(Y_train, axis=1))
test_accuracy = np.mean(test_predictions == np.argmax(Y_test, axis=1))

print(f"Train Accuracy: {train_accuracy * 100}%")
print(f"Test Accuracy: {test_accuracy * 100}%")


AL shape: (5, 250)
Y shape: (250, 5)
Cost after iteration 0: 80.469316
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL s

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
Cost after iteration 900: 80.467159
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL

...

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
Cost after iteration 2000: 80.453851
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A

AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
AL shape: (5, 250)
Y shape: (250, 5)
A